## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [2]:
!pip list

Package                          Version
-------------------------------- -----------------------
absl-py                          0.10.0
astunparse                       1.6.3
async-generator                  1.10
attrs                            20.3.0
backcall                         0.2.0
bleach                           3.2.1
cachetools                       4.1.1
certifi                          2020.11.8
chardet                          3.0.4
click                            7.1.2
cloudpickle                      1.6.0
cycler                           0.10.0
dbus-python                      1.2.16
decorator                        4.4.2
defusedxml                       0.6.0
dill                             0.3.3
distro                           1.4.0
easydict                         1.9
entrypoints                      0.3
faiss-gpu                        1.6.5
filelock                         3.0.12
Flask                            1.1.2
future                           0.18.2


In [3]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
    # np.mean(matrix, int) 後面加int 會得到平均數，而加axis=(int)則會得到array
    # 不加得到所有數平均數
    mean = np.mean(X_train,axis=(0,1,2,3))
    std = np.std(X_train, axis=(0, 1, 2, 3))
    X_train = (X_train-mean)/(std+1e-7)
    X_test = (X_test-mean)/(std+1e-7) 
    return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

170500096/170498071 [==============================] - 28s 0us/step
(50000, 32, 32, 3)


In [5]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [71]:
classifier = Sequential()

#卷積組合1
classifier.add(Convolution2D(32,kernel_size=(3,3),input_shape=(32,32,3),padding='same',activation='relu')) #32,3,3,input_shape=(32,32,3),activation='relu'
classifier.add(BatchNormalization())
'''自己決定MaxPooling2D放在哪裡'''
# classifier.add(MaxPooling2D(pool_size=(2,2)))


#卷積組合2
classifier.add(Convolution2D(32,kernel_size=(3,3),padding='same',activation='relu'))
classifier.add(BatchNormalization())
'''自己決定MaxPooling2D放在哪裡'''
# classifier.add(MaxPooling2D(pool_size=(2,2)))


#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(units=200,activation='relu')) #output_dim=100,activation=relu
#100 0.1401
#200 0.0940
#400 0.0985

#輸出
classifier.add(Dense(units=10,activation='softmax'))

# classifier.summary()

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=64,epochs=100)
#Batch
#16    0.9139
#32    0.8954
#64    0.8999 VV
#128   0.9255
#256   0.9737
#512   1.0215
#1024  1.1228
#2048  1.2292
#4096  1.3598
#8192  1.5475
#16384 1.6704
# kernel 5,5 5,5 0.1969
# kernel 3,3 5,5 0.2053

Epoch 1/100
782/782 [==============================] - 2s 2ms/step - loss: 1.5528 - accuracy: 0.4951
Epoch 2/100
782/782 [==============================] - 2s 2ms/step - loss: 0.9497 - accuracy: 0.6666
Epoch 3/100
782/782 [==============================] - 2s 2ms/step - loss: 0.6802 - accuracy: 0.7607
Epoch 4/100
782/782 [==============================] - 2s 2ms/step - loss: 0.4552 - accuracy: 0.8418
Epoch 5/100
782/782 [==============================] - 2s 2ms/step - loss: 0.2973 - accuracy: 0.8980
Epoch 6/100
782/782 [==============================] - 2s 2ms/step - loss: 0.2083 - accuracy: 0.9286
Epoch 7/100
782/782 [==============================] - 2s 2ms/step - loss: 0.1621 - accuracy: 0.9458
Epoch 8/100
782/782 [==============================] - 2s 2ms/step - loss: 0.1373 - accuracy: 0.9543
Epoch 9/100
782/782 [==============================] - 2s 2ms/step - loss: 0.1212 - accuracy: 0.9597
Epoch 10/100
782/782 [==============================] - 2s 2ms/step - loss: 0.1027 - accura

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [107]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
# print(input_example)
classifier.predict(input_example)

array([[1.6050570e-03, 2.8003274e-07, 9.5403248e-01, 4.4304922e-02,
        1.9453682e-06, 4.0892566e-05, 2.3434133e-08, 1.6233915e-06,
        1.2803923e-05, 2.2939009e-08]], dtype=float32)